In [2]:
from src.data_structures import Instance
from src.solvers.OG.model import CustomSolver,Net
from src.solvers.OG.integer_quantizer import KmeansQuantizer
import numpy as np
import torch
from matplotlib import pyplot as plt
from functools import partial
import torch.nn as nn
import torch.optim as optim
import sys
torch.set_default_tensor_type(torch.DoubleTensor)


In [3]:
instance = Instance.generate(100,10)

In [4]:
solver = CustomSolver(instance=instance)

In [33]:
from src.solvers.OG.integer_quantizer import EmbeddingQuantizer

benefit_data = solver.benefit_data
#quantizer = KmeansQuantizer(benefit_data,100)
quantizer = EmbeddingQuantizer(benefit_data,1,20,1000,1)
solver.set_integer_quantizer(quantizer)
print(solver.quantizer.cluster_centers())

124.96779143381195
tensor([47.8174, -3.8432, 43.8746, -4.9404, 29.5989, -3.9260, -3.0228, -3.8614,
        -7.3067,  6.9368, -4.7404, -5.7356, -5.4328, 22.3152, 38.9871,  5.7048,
        -3.9408,  1.2738, 21.1678, -4.2835])


In [6]:
x = solver.create_vectorized_data()
x

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [7]:
y = solver.quantizer.indexes().view(-1,1)
#Normalizo y 
y_size = torch.max(y)+1

def tupla_a_vector_binario(tupla,size):
    output = np.zeros(size)
    for i in tupla:
        output[i] = 1
    return output

y = torch.tensor(np.array(list(map(partial(tupla_a_vector_binario,size=y_size),y)),dtype=np.float32))





En base a la definicion anterior estoy intendando predecir el indice y, en base a la tupla polinomial vectorizada x

In [8]:
#Intentar adivinar el indice

net = Net(solver.instance.n_items,y_size)
criterion = nn.SmoothL1Loss()
optimizer = optim.SGD(net.parameters(),lr = 1)

dataset = torch.utils.data.TensorDataset(x,y)
dataloader = torch.utils.data.DataLoader(dataset,batch_size=3,shuffle=True)

In [9]:
for epoch in range(100):
    total_loss = 0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        batch_output = net(batch_x)
        batch_loss = criterion(batch_output, batch_y)
        batch_loss.backward()
        optimizer.step()
        total_loss += batch_loss.item()
    
    sys.stdout.write(f'\rEpoch {epoch}, Loss {total_loss / len(dataloader)}')
    sys.stdout.flush()

Epoch 1, Loss 0.015238588406104456

Epoch 99, Loss 0.0030656603433440893

In [10]:
#Ahora uso la red para calcular el beneficio aproximado de cada item
canon = torch.eye(instance.n_items)
vectorized_predicted_indexes = net(canon)
predicted_indexes = torch.argmax(vectorized_predicted_indexes,dim=1)

predicted_value_per_item = torch.index_select(solver.quantizer.cluster_centers(), 0, predicted_indexes)

In [11]:
predicted_value_per_item.T

/tmp/ipykernel_83709/1670099730.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /build/python-pytorch/src/pytorch-opt/aten/src/ATen/native/TensorShape.cpp:3571.)
  predicted_value_per_item.T


tensor([3.9314, 8.9468, 3.9314, 3.9314, 3.9314, 3.9314, 8.9468, 8.9468, 3.9314,
        3.9314, 3.9314, 8.9468, 8.9468, 3.9314, 8.9468, 3.9314, 3.9314, 3.9314,
        3.9314, 3.9314, 3.9314, 3.9314, 3.9314, 3.9314, 8.9468, 3.9314, 8.9468,
        3.9314, 3.9314, 3.9314, 3.9314, 8.9468, 3.9314, 3.9314, 8.9468, 3.9314,
        3.9314, 3.9314, 3.9314, 3.9314, 8.9468, 3.9314, 3.9314, 8.9468, 3.9314,
        8.9468, 8.9468, 8.9468, 8.9468, 8.9468, 8.9468, 3.9314, 3.9314, 8.9468,
        3.9314, 8.9468, 3.9314, 3.9314, 3.9314, 3.9314, 3.9314, 3.9314, 3.9314,
        3.9314, 8.9468, 8.9468, 3.9314, 8.9468, 3.9314, 8.9468, 3.9314, 3.9314,
        3.9314, 3.9314, 3.9314, 8.9468, 3.9314, 3.9314, 3.9314, 3.9314, 3.9314,
        3.9314, 3.9314, 3.9314, 8.9468, 3.9314, 3.9314, 8.9468, 3.9314, 3.9314,
        8.9468, 3.9314, 3.9314, 3.9314, 3.9314, 8.9468, 8.9468, 8.9468, 3.9314,
        8.9468])

In [12]:
from src.solvers.collection import SolverCollection,Solution
profits = torch.tensor(instance.profits).view(-1,1)
profits = (profits+predicted_value_per_item).T.detach().numpy()[0]
instance2 = Instance(instance.n_items,instance.gamma,instance.budget,profits,instance.costs,dict())

In [13]:
base_instance = Instance(instance.n_items,instance.gamma,instance.budget,instance.profits,instance.costs,dict())
base_sol = SolverCollection.gurobi_optimal(base_instance)
print(base_sol)

Sol(of:2790.250103133364,time:0.04005885124206543)


In [14]:
#Sol with predicted values


sol2 = SolverCollection.gurobi_optimal(instance2)
o = instance.evaluate(sol2.sol)
final_sol = Solution(o,sol2.sol,sol2.time)
print(final_sol)


Sol(of:3391.2562244007863,time:0.04061722755432129)


In [15]:
optimal_sol = SolverCollection.gurobi_optimal(instance)
print(optimal_sol)

Sol(of:3653.646907688214,time:0.5654070377349854)


In [16]:
baldo_sol = SolverCollection.baldo_ML(instance)
print(baldo_sol)

/usr/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.0 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.3.0 when using version 1.3.1. This might lead to breaking code or in

Sol(of:3653.646907688214,time:0.7768564224243164)


In [17]:
ga_sol = SolverCollection.baldo_GA(instance)
print(ga_sol)

Sol(of:3576.09511422617,time:0.4122023582458496)
